# Лабораторная 2

### Английский датасет

Используем модель insults_kaggle_bert для определения тональности в англ тексте

In [1]:
from deeppavlov import build_model
model = build_model('insults_kaggle_bert', download=True, install=True)

2025-03-02 14:54:39.224 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/insults_kaggle_torch_bert_v5.tar.gz download because of matching hashes
2025-03-02 14:54:39.864 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/datasets/insults_data.tar.gz download because of matching hashes
D:\gadd\envs\Brp2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\gadd\envs\Brp2\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializi

Читаем и выводим первые 15 строк датасета

In [2]:
import pandas as pd
tabl = pd.read_csv('./toxic_comments.csv', 
    engine='python', 
    on_bad_lines='skip',
)
tabl.head(15)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


### Предобработка

1. Удаляем лишний столбец

In [3]:
tabl = tabl.drop('Unnamed: 0', axis=1)

In [4]:
!pip install pymorphy2

Применяем лемматизацию и векторизацию

In [5]:
import re
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer

Векторизация

In [6]:
vec = TfidfVectorizer()
def vctr(series: pd.Series) -> list[str]:
    return list(vec.fit_transform(series))
tabl['vector'] = vctr(tabl['text'])

Лемматизация

In [7]:
lem = pymorphy2.MorphAnalyzer()

def cls(series: pd.Series) -> pd.Series:
    return series.apply(cl)
def cl(text: str) -> str:
    return clear(lemm(text))
def clear(text: str) -> str:
    return re.sub(r'[^\w\s]', ' ', text)
def lemm(text: str) -> str:
    return ' '.join(lem.parse(word)[0].normal_form for word in text.lower().split())
    
tabl['lemText'] = cls(tabl['text'])

Смотрим результат

In [8]:
tabl.head(15)

,text,toxic,vector,lemText
0,Explanation\nWhy the edits made under my usern...,0,"(0, 3446)\t0.16769576946881884\n (0, 4373)\...",explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,"(0, 142295)\t0.16412463608924402\n (0, 2613...",d aww he matches this background colour i m s...
2,"Hey man, I'm really not trying to edit war. It...",0,"(0, 69411)\t0.20448767581830993\n (0, 8935)...",hey man i m really not trying to edit war it...
3,"""\nMore\nI can't make any real suggestions on ...",0,"(0, 137467)\t0.16225009518550423\n (0, 5929...",more i can t make any real suggestions on im...
4,"You, sir, are my hero. Any chance you remember...",0,"(0, 146787)\t0.18634179087764524\n (0, 1136...",you sir are my hero any chance you remember...
5,"""\n\nCongratulations from me as well, use the ...",0,"(0, 136592)\t0.4950649603572783\n (0, 14205...",congratulations from me as well use the too...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,"(0, 149014)\t0.2862025416767899\n (0, 15265...",cocksucker before you piss around on my work
7,Your vandalism to the Matt Shirvington article...,0,"(0, 18797)\t0.2916301922744916\n (0, 148118...",your vandalism to the matt shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,"(0, 1661)\t0.1733668525569962\n (0, 682)\t0...",sorry if the word nonsense was offensive to ...
9,alignment on this subject and which are contra...,0,"(0, 44979)\t0.6231252205333644\n (0, 135212...",alignment on this subject and which are contra...


Добавляем новый столбец с результатами предсказаний нашей модели. 

Так как не хватает памяти для обработки всего датасета, мы копируем первые 1000 строк в новую таблицу и работаем с ними.

In [9]:
tabl2 = tabl.head(1000).copy()
tabl2['result'] = model(tabl2['lemText'].tolist())
def forma(series):
    series = series.str.replace('Not Insult', '0').str.replace('Insult', '1').astype(int)
    return series
tabl2['result'] = forma(tabl2['result'])
tabl.loc[:999, 'result'] = tabl2['result']

Итог работы:

In [10]:
tabl2.head(10)

,text,toxic,vector,lemText,result
0,Explanation\nWhy the edits made under my usern...,0,"(0, 3446)\t0.16769576946881884\n (0, 4373)\...",explanation why the edits made under my userna...,0
1,D'aww! He matches this background colour I'm s...,0,"(0, 142295)\t0.16412463608924402\n (0, 2613...",d aww he matches this background colour i m s...,0
2,"Hey man, I'm really not trying to edit war. It...",0,"(0, 69411)\t0.20448767581830993\n (0, 8935)...",hey man i m really not trying to edit war it...,0
3,"""\nMore\nI can't make any real suggestions on ...",0,"(0, 137467)\t0.16225009518550423\n (0, 5929...",more i can t make any real suggestions on im...,0
4,"You, sir, are my hero. Any chance you remember...",0,"(0, 146787)\t0.18634179087764524\n (0, 1136...",you sir are my hero any chance you remember...,0
5,"""\n\nCongratulations from me as well, use the ...",0,"(0, 136592)\t0.4950649603572783\n (0, 14205...",congratulations from me as well use the too...,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,"(0, 149014)\t0.2862025416767899\n (0, 15265...",cocksucker before you piss around on my work,1
7,Your vandalism to the Matt Shirvington article...,0,"(0, 18797)\t0.2916301922744916\n (0, 148118...",your vandalism to the matt shirvington article...,1
8,Sorry if the word 'nonsense' was offensive to ...,0,"(0, 1661)\t0.1733668525569962\n (0, 682)\t0...",sorry if the word nonsense was offensive to ...,0
9,alignment on this subject and which are contra...,0,"(0, 44979)\t0.6231252205333644\n (0, 135212...",alignment on this subject and which are contra...,0
